In [10]:
from tkinter import *
from tkinter import ttk
import nbimporter
import  math
import threading
import time
import sys
if('..' not in sys.path):
    sys.path.insert(0,'..')
    
from Maze.Maze import Maze
from Maths.Cord import Cord

class MainWindow:
    agentsFinished=0
    
    def __init__(self,m, a, t):
        self.maze=m
        self.agents=a
        self.time=t
        self.displayedTime=t
        self.mPlaying=False
        
    def display(self):
        t1=threading.Thread(target=self.createAndShowGUI)
        t1.start()
        #self.createAndShowGUI()
        
    def createAndShowGUI(self):
        self.root=Tk()
        self.root.title("Main Frame")
        content = ttk.Frame(self.root)
        self.mapWidth=500
        self.mapHeight=500
        self.frame = Canvas(content, width=self.mapWidth, height=self.mapHeight)
        self.addMapPanel(self.frame)
        namelbl = ttk.Label(content, text="Name")
        name = ttk.Entry(content)

        onevar = BooleanVar()
        twovar = BooleanVar()
        threevar = BooleanVar()
        onevar.set(True)
        twovar.set(False)
        threevar.set(True)

        back = ttk.Button(content, text="Back")
        back.bind('<Button-1>',self.moveBack)
        play = ttk.Button(content, text="Play")
        #play.bind('<Button-1>',self.runOrPause)
        forward = ttk.Button(content, text="Forward")
        forward.bind('<Button-1>',self.moveForward)
        optimal = ttk.Button(content, text="Optimal")
        optimal.bind('<Button-1>',self.addOptRoutes)
        close = ttk.Button(content, text="Close")
        close.bind('<Button-1>',self.close)

        content.grid(column=0, row=0)
        self.frame.grid(column=0, row=0, columnspan=3, rowspan=2)
        namelbl.grid(column=3, row=0, columnspan=2)
        name.grid(column=3, row=1, columnspan=2)
        back.grid(column=0, row=3)
        play.grid(column=1, row=3)
        forward.grid(column=2, row=3)
        optimal.grid(column=3, row=3)
        close .grid(column=4, row=3)

        self.root.mainloop()

    def addMapPanel(self, canvas):
        self.routesDisplayed=False        
        self.displayMaze(canvas, self.displayedTime)
        
    def displayMaze(self,canvas, t0):
        placement=[]
        for a in self.agents:
            if (a.getState()):
                aHis=a.getHistory()
                position=aHis[t0]
                placement.append(position)
            else:
                position=a.getPos()
                placement.append(position)
            his=""
            for c1 in a.getHistory():
                his+=c1.CordToString()+", "
            #print(a.getState()," position: "+position.CordToString(), a.getName(), self.displayedTime, t0, len(placement))
            #print(a.getName(),his)    
        xwidth= math.floor((self.mapWidth)/self.maze.getWidth())  
        ywidth= math.floor((self.mapHeight)/self.maze.getHeight())  
        squaresize=min(xwidth,ywidth)
        
        for a in range(0,self.mapWidth+squaresize,squaresize):
            for b in range(0,self.mapHeight+squaresize,squaresize):
                if(self.maze.getValue(int(a/squaresize),int(b/squaresize))==1):
                    canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="black")
                else:
                    canvas.create_rectangle(b,a,b+squaresize,a+squaresize, fill="white")
                    
        if(self.routesDisplayed):
            routes=self.maze.GetOptimalRoute()
            for route in routes:
                for c in range(0,self.mapWidth+squaresize,squaresize):
                    for d in range(0,self.mapHeight+squaresize,squaresize):
                        if(Cord(int(c/squaresize),int(d/squaresize))in route):
                            canvas.create_rectangle(d,c,d+squaresize,c+squaresize, fill="red")
          
        
        for e in range(0,self.mapWidth+squaresize,squaresize):
            for f in range(0,self.mapHeight+squaresize,squaresize):
                if(Cord(int(e/squaresize),int(f/squaresize)) in placement):
                    canvas.create_rectangle(f,e,f+squaresize,e+squaresize, fill="blue")                   
        
        for x in range(0,self.mapWidth+squaresize,squaresize):
            canvas.create_line(x, 0, x, self.maze.getHeight()*squaresize, fill="#476042")
        for y in range(0,self.mapHeight+squaresize,squaresize):
            canvas.create_line(0, y, self.maze.getWidth()*squaresize, y, fill="#476042")
            
    def close(self,event):
        self.root.destroy()
        
    def addOptRoutes(self, event):
        if(self.routesDisplayed):
            self.routesDisplayed=False
        else:
            self.routesDisplayed=True
        self.displayMaze(self.frame, self.displayedTime)
        
    def moveForward(self,event=None):
        if(self.displayedTime==self.time):
            for a in self.agents:
                a.NextMove()
                if(a.getPos() in self.maze.getExit()):
                    if(a.getState()):
                        a.stop()
                        MainWindow.agentsFinished+=1
                if(MainWindow.agentsFinished==len(self.agents)):
                    self.mPlaying=False            
            self.displayedTime+=1
            self.time+=1
            self.displayMaze(self.frame, self.displayedTime)
            
        elif(self.displayedTime<self.time):
            self.displayedTime+=1
            self.displayMaze(self.frame, self.displayedTime)
            
    def moveBack(self,event):
        if (self.mPlaying):
            pass#self.runOrPause()
        if(self.displayedTime==0):
            self.displayMaze(self.frame, self.displayedTime)
            return
        self.displayedTime-=1
        self.displayMaze(self.frame, self.displayedTime)
        
    def runOrPause(self,event):
        if(self.mPlaying):      
            self.mPlaying=False
        else:     
            self.mPlaying=True
            while self.mPlaying:
                time.sleep(0.5)
                self.moveForward()
                

In [11]:
m= Maze("Test,10,10,0,4,9,4,1111211111100000000111100110011000000001100001101111011000111000000001100101010110000000011111311111")
#m= Maze("Test,3,4,0,1,2,1,121101101131")

mw=MainWindow(m,[],0)
mw.display()